# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [2]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [3]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [4]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [18]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [19]:
players_180_190 = player_data[(
    player_data.height >= 180) & (player_data.height <= 190)]

In [22]:
players_180_190.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
5,6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161
10,11,23889,Aaron Mokoena,47189,1980-11-25 00:00:00,182.88,181


**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [20]:
players_1980 = player_data[pd.DatetimeIndex(player_data.birthday).year == 1980]


In [69]:
players_180_190.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
5,6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161
10,11,23889,Aaron Mokoena,47189,1980-11-25 00:00:00,182.88,181


**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [33]:
highest_players = player_data.sort_values(['height', 'player_name'], ascending=[False, True]).head(10)

In [34]:
highest_players.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
5901,5908,148325,Kristof van Hout,185306,1987-02-09 00:00:00,208.28,243
1298,1301,150209,Bogdan Milic,189967,1987-11-24 00:00:00,203.20,216
1923,1926,103428,Costel Pantilimon,192613,1987-02-01 00:00:00,203.20,212
3270,3274,601304,Fejsal Mulic,226114,1994-10-03 00:00:00,203.20,185
5561,5568,26585,Jurgen Wevers,148875,1979-01-12 00:00:00,203.20,212


**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [62]:
years_born_players = [(year, count) for year, count in player_data.groupby([pd.DatetimeIndex(player_data.birthday).year]).size().iteritems()]


In [63]:
print(years_born_players)

[(1967, 1), (1968, 1), (1969, 6), (1970, 12), (1971, 12), (1972, 22), (1973, 43), (1974, 77), (1975, 113), (1976, 156), (1977, 191), (1978, 263), (1979, 345), (1980, 434), (1981, 475), (1982, 487), (1983, 541), (1984, 607), (1985, 593), (1986, 710), (1987, 722), (1988, 802), (1989, 723), (1990, 696), (1991, 699), (1992, 643), (1993, 529), (1994, 462), (1995, 331), (1996, 234), (1997, 103), (1998, 24), (1999, 3)]


**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [85]:
adriano_mean, adriano_std = (
    player_data[player_data.player_name.str.startswith('Adriano')]['height'].mean(),
    player_data[player_data.player_name.str.startswith('Adriano')]['height'].std()
)


In [81]:
print(f"adriano_mean is {adriano_mean} and adriano_std is {adriano_std}")

adriano_mean is 182.118 and adriano_std is 5.361479480723788


**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [144]:
dow_with_min_players_born = player_data.groupby([pd.DatetimeIndex(player_data.birthday).dayofweek]).size().idxmin()

In [143]:
print(dow_with_min_players_born)

6


**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [166]:
league_most_matches_data = pd.read_sql("SELECT name, count FROM League INNER JOIN (SELECT league_id, Count(*) as count from Match GROUP BY Match.league_id) as M ON League.id = M.league_id;", db)

In [167]:
league_most_matches = league_most_matches_data.sort_values(['count', 'name'], ascending=[False, True]).iloc[0]['name']

In [168]:
print(league_most_matches)

England Premier League


**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [6]:
max_matches_player_data = pd.read_sql(" UNION ALL ".join([f"SELECT id, home_player_{x} as player FROM Match WHERE home_player_{x} NOT NULL" for x in range(1, 11)] + [f"SELECT id, away_player_{x} as player FROM Match WHERE away_player_{x} NOT NULL" for x in range(1, 11)]), db)
max_matches_player = player_data[player_data.player_api_id == max_matches_player_data.groupby('player').size().idxmax()].iloc[0]["player_name"]

In [7]:
print(max_matches_player)

Steve Mandanda


**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [185]:
top_correlated_features__data = pd.read_sql("SELECT * FROM Player_Attributes", db)
top_correlated_features__exclude_columns = ['id', 'player_api_id', 'player_fifa_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
top_correlated_features__corr = top_correlated_features__data.loc[:, ~top_correlated_features__data.columns.isin(top_correlated_features__exclude_columns)].corr().unstack().drop_duplicates().sort_values(ascending=False)
top_correlated_features__corr = top_correlated_features__corr[top_correlated_features__corr.index.get_level_values(0) != top_correlated_features__corr.index.get_level_values(1)]
#answer
top_correlated_features = top_correlated_features__corr.head(5)



In [186]:
print(top_correlated_features.index.values)

[('gk_positioning', 'gk_reflexes') ('gk_handling', 'gk_reflexes')
 ('gk_handling', 'gk_positioning') ('standing_tackle', 'sliding_tackle')
 ('marking', 'standing_tackle')]


**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [ ]:
neymar_similarities = ()

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [ ]:
borussia_bundesliga_2008_2009_matches = ()

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [ ]:
team_most_matches_bundesliga_2008_2009 = ()

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [209]:
arsenal_won_matches_2015_2016_data = pd.read_sql("SELECT Count() as count FROM Team INNER JOIN (Select match_api_id, date, home_team_api_id as team_api_id, (home_team_goal - away_team_goal > 0) as won FROM Match UNION Select match_api_id, date, away_team_api_id as team_api_id, (away_team_goal - home_team_goal > 0) as won FROM Match) as M on Team.team_api_id = M.team_api_id WHERE team_short_name = 'ARS' and won = 1 and date BETWEEN '2015-01-01' and '2016-12-31';", db)
arsenal_won_matches_2015_2016 = arsenal_won_matches_2015_2016_data.iloc[0, 0]

In [210]:
print(arsenal_won_matches_2015_2016)

33


**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [ ]:
team_highest_winrate_2015_2016 = ()

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [ ]:
highest_gap_england_2010_2011 = ()

### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")